# 合并数据集：合并与连接

`Pandas` 的基本特性之一就是高性能的内存式数据连接（`join`）与合并（`merge`）操作。  
如果你有使用数据库的经验，那么对这类操作一定很熟悉。  
`Pandas` 的主接口是 `pd.merge` 函数，下面让我们通过一些示例来介绍它的用法。

In [1]:
import numpy as np
import pandas as pd

In [2]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 1. 关系代数

`pd.merge()` 实现的功能基于**关系代数**（`relational algebra`）的一部分。  
关系代数是处理关系型数据的通用理论，绝大部分数据库的可用操作都以此为理论基础。  
关系代数方法论的强大之处在于，它提出的若干简单操作规则经过组合就可以为任意数据集构建十分复杂的操作。  
借助在数据库或程序里已经高效实现的基本操作规则，你可以完成许多非常复杂的操作。  
`Pandas` 在 `pd.merge()` 函数与 `Series` 和 `DataFrame` 的 `join()` 方法里实现了这些基本操作规则。  
下面来看看如何用这些简单的规则连接不同数据源的数据。

## 2. 数据连接的类型

`pd.merge()` 函数实现了三种数据连接的类型：**一对一**、**多对一**和**多对多**。  
这三种数据连接类型都通过 `pd.merge()` 接口进行调用，根据不同的数据连接需求进行不同的操作。

### 2.1. 一对一连接

一对一连接可能是最简单的数据合并类型了。

In [3]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
df3 = pd.merge(df1, df2)  # pd.merge() 方法会发现两个 DataFrame 都有“employee”列，并会自动以这列作为键进行连接。两个输入的合并结果是一个新的 DataFrame。
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 2.2. 多对一连接

多对一连接是指，在需要连接的两个列中，有一列的值有重复。通过多对一连接获得的结果 DataFrame 将会保留重复值。

### 2.3. 多对多连接

多对多连接是个有点儿复杂的概念，不过也可以理解。如果左右两个输入的共同列都包含重复值，那么合并的结果就是一种多对多连接。